In [104]:
from pprint import pprint 

# !pip install pymongo
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27018/?directConnection=true")

### Налаштувати реплікацію в конфігурації: 

Primary with Two Secondary Members (всі ноди можуть бути запущені як окремі процеси або у Docker контейнерах) - http://docs.mongodb.org/manual/core/replica-set-architecture-three-members/ 

Deploy a Replica Set for Testing and Development- http://docs.mongodb.org/manual/tutorial/deploy-replica-set-for-testing/
http://www.tugberkugurlu.com/archive/setting-up-a-mongodb-replica-set-with-docker-and-connecting-to-it-with-a--net-core-app 


In [98]:
myclient.admin.command("replSetGetStatus")

{'set': 'rs0',
 'date': datetime.datetime(2024, 3, 20, 20, 49, 59, 650000),
 'myState': 1,
 'term': 1,
 'syncSourceHost': '',
 'syncSourceId': -1,
 'heartbeatIntervalMillis': 2000,
 'majorityVoteCount': 2,
 'writeMajorityCount': 2,
 'votingMembersCount': 3,
 'writableVotingMembersCount': 3,
 'optimes': {'lastCommittedOpTime': {'ts': Timestamp(1710967783, 1), 't': 1},
  'lastCommittedWallTime': datetime.datetime(2024, 3, 20, 20, 49, 43, 821000),
  'readConcernMajorityOpTime': {'ts': Timestamp(1710967783, 1), 't': 1},
  'appliedOpTime': {'ts': Timestamp(1710967783, 1), 't': 1},
  'durableOpTime': {'ts': Timestamp(1710967783, 1), 't': 1},
  'lastAppliedWallTime': datetime.datetime(2024, 3, 20, 20, 49, 43, 821000),
  'lastDurableWallTime': datetime.datetime(2024, 3, 20, 20, 49, 43, 821000)},
 'lastStableRecoveryTimestamp': Timestamp(1710967783, 1),
 'electionCandidateMetrics': {'lastElectionReason': 'electionTimeout',
  'lastElectionDate': datetime.datetime(2024, 3, 20, 20, 8, 0, 674000),


### 2. Продемонструвати Read Preference Modes: читання з primary і secondary node (http://docs.mongodb.org/manual/core/read-preference/)

In [79]:
myclient.read_preference

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000001'), 'opTime': {'ts': Timestamp(1710967438, 1), 't': 1}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1710967438, 1), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1710967438, 1)}, acknowledged=True)

### 3. Спробувати зробити запис з однією відключеною нодою та write concern рівнім 3 та нескінченім таймаутом. Спробувати під час таймаута включити відключену ноду 

In [64]:
import time

mydb = myclient["dd"]
items = mydb.get_collection('items', write_concern=pymongo.WriteConcern(w=3, wtimeout=0))

start_time = time.time()

print(items.insert_one({ 'item': 1, 'qty' : 1000, 'type': "Clasper" }))

print(round(time.time() - start_time, 2), ' seconds')

InsertOneResult(ObjectId('65fb49643836103bf9b3429c'), acknowledged=True)
19.5  seconds


### 4. Аналогічно попередньому пункту, але задати скінченний таймаут та дочекатись його закінчення. Перевірити чи данні записались і чи доступні на читання з рівнем readConcern: “majority”

#### Відповідь - так, доступні

In [88]:
items.delete_many({'item': 2})

items = mydb.get_collection('items', write_concern=pymongo.WriteConcern(w=3, wtimeout=10000))

start_time = time.time()

print(items.insert_one({ 'item': 2, 'qty' : 1000, 'type': "Clasper" }))

print(round(time.time() - start_time, 2), ' seconds')

WTimeoutError: waiting for replication timed out, full error: {'code': 64, 'codeName': 'WriteConcernFailed', 'errmsg': 'waiting for replication timed out', 'errInfo': {'wtimeout': True, 'writeConcern': {'w': 3, 'wtimeout': 10000, 'provenance': 'clientSupplied'}}}

In [97]:
items = mydb.get_collection('items', read_concern=pymongo.read_concern.ReadConcern(level='majority'))

pprint(items.find_one({'item': 2}))

{'_id': ObjectId('65fb4b973836103bf9b342a2'),
 'item': 2,
 'qty': 1000,
 'type': 'Clasper'}


### 5. Продемонстрував перевибори primary node відключивши поточний primary (Replica Set Elections) - http://docs.mongodb.org/manual/core/replica-set-elections/  
і що після відновлення роботи старої primary на неї реплікуються нові дані, які з'явилися під час її простою


In [99]:
# Перевіремо PRIMARY
myclient.admin.command("replSetGetStatus")
# Це mongo-node1:27017

{'set': 'rs0',
 'date': datetime.datetime(2024, 3, 20, 20, 50, 15, 117000),
 'myState': 1,
 'term': 1,
 'syncSourceHost': '',
 'syncSourceId': -1,
 'heartbeatIntervalMillis': 2000,
 'majorityVoteCount': 2,
 'writeMajorityCount': 2,
 'votingMembersCount': 3,
 'writableVotingMembersCount': 3,
 'optimes': {'lastCommittedOpTime': {'ts': Timestamp(1710967810, 1), 't': 1},
  'lastCommittedWallTime': datetime.datetime(2024, 3, 20, 20, 50, 10, 957000),
  'readConcernMajorityOpTime': {'ts': Timestamp(1710967810, 1), 't': 1},
  'appliedOpTime': {'ts': Timestamp(1710967810, 1), 't': 1},
  'durableOpTime': {'ts': Timestamp(1710967810, 1), 't': 1},
  'lastAppliedWallTime': datetime.datetime(2024, 3, 20, 20, 50, 10, 957000),
  'lastDurableWallTime': datetime.datetime(2024, 3, 20, 20, 50, 10, 957000)},
 'lastStableRecoveryTimestamp': Timestamp(1710967783, 1),
 'electionCandidateMetrics': {'lastElectionReason': 'electionTimeout',
  'lastElectionDate': datetime.datetime(2024, 3, 20, 20, 8, 0, 674000),


In [105]:
# Виключимо PRIMARY
myclient.admin.command("replSetGetStatus")

{'set': 'rs0',
 'date': datetime.datetime(2024, 3, 20, 21, 1, 37, 72000),
 'myState': 1,
 'term': 2,
 'syncSourceHost': '',
 'syncSourceId': -1,
 'heartbeatIntervalMillis': 2000,
 'majorityVoteCount': 2,
 'writeMajorityCount': 2,
 'votingMembersCount': 3,
 'writableVotingMembersCount': 3,
 'optimes': {'lastCommittedOpTime': {'ts': Timestamp(1710968494, 1), 't': 2},
  'lastCommittedWallTime': datetime.datetime(2024, 3, 20, 21, 1, 34, 923000),
  'readConcernMajorityOpTime': {'ts': Timestamp(1710968494, 1), 't': 2},
  'appliedOpTime': {'ts': Timestamp(1710968494, 1), 't': 2},
  'durableOpTime': {'ts': Timestamp(1710968494, 1), 't': 2},
  'lastAppliedWallTime': datetime.datetime(2024, 3, 20, 21, 1, 34, 923000),
  'lastDurableWallTime': datetime.datetime(2024, 3, 20, 21, 1, 34, 923000)},
 'lastStableRecoveryTimestamp': Timestamp(1710968454, 1),
 'electionCandidateMetrics': {'lastElectionReason': 'stepUpRequestSkipDryRun',
  'lastElectionDate': datetime.datetime(2024, 3, 20, 20, 51, 14, 8730

In [107]:
# Додамо докемунтів
mydb = myclient["dd"]
items = mydb.get_collection('items', write_concern=pymongo.WriteConcern(w=3, wtimeout=6000))

print(items.insert_many([{ 'item': 3, 'qty' : 1000, 'type': "Clasper" },
                         { 'item': 4, 'qty' : 1000, 'type': "Clasper" },
                         { 'item': 5, 'qty' : 1000, 'type': "Clasper" }]))

BulkWriteError: batch op errors occurred, full error: {'writeErrors': [], 'writeConcernErrors': [{'code': 64, 'codeName': 'WriteConcernFailed', 'errmsg': 'waiting for replication timed out', 'errInfo': {'wtimeout': True, 'writeConcern': {'w': 3, 'wtimeout': 6000, 'provenance': 'clientSupplied'}}}], 'nInserted': 3, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}

In [114]:
previous_primary_node = pymongo.MongoClient("mongodb://localhost:27017/?directConnection=true")
mydb = previous_primary_node["dd"]
items = mydb.get_collection('items', read_concern=pymongo.read_concern.ReadConcern(level='local'))

pprint(items.find_one({'item': 4}))
pprint(items.find_one({'item': 3}))
pprint(items.find_one({'item': 5}))

{'_id': ObjectId('65fb4f6d3836103bf9b342a7'),
 'item': 4,
 'qty': 1000,
 'type': 'Clasper'}
{'_id': ObjectId('65fb4f6d3836103bf9b342a6'),
 'item': 3,
 'qty': 1000,
 'type': 'Clasper'}
{'_id': ObjectId('65fb4f6d3836103bf9b342a8'),
 'item': 5,
 'qty': 1000,
 'type': 'Clasper'}


### 6. Привести кластер до неконсистентного стану користуючись моментом часу коли primary node не одразу помічає відсутність secondary node 
- відключивши дві secondary node, протягом 5 сек. на мастер записати значення (з w:1) і перевірити, що воно записалось
- спробувати зчитати це значення з різними рівнями read concern - readConcern: {level: <"majority"|"local"| "linearizable">} - значення має бути доступне тільки при читанні з рівнем “local”
- включити дві інші ноди таким чином, щоб вони не бачили попереднього мастера (його можна відключити) і дочекатись поки вони оберуть нового мастера 
- підключити (включити) попередню primary-ноду до кластеру і подивитись, що сталось зі значенням яке було на неї записано - воно має зникнути


In [117]:
# Перевіремо PRIMARY
myclient.admin.command("replSetGetStatus")
# Це mongo-node2:27017

{'set': 'rs0',
 'date': datetime.datetime(2024, 3, 20, 21, 11, 28, 44000),
 'myState': 1,
 'term': 2,
 'syncSourceHost': '',
 'syncSourceId': -1,
 'heartbeatIntervalMillis': 2000,
 'majorityVoteCount': 2,
 'writeMajorityCount': 2,
 'votingMembersCount': 3,
 'writableVotingMembersCount': 3,
 'optimes': {'lastCommittedOpTime': {'ts': Timestamp(1710969084, 1), 't': 2},
  'lastCommittedWallTime': datetime.datetime(2024, 3, 20, 21, 11, 24, 949000),
  'readConcernMajorityOpTime': {'ts': Timestamp(1710969084, 1), 't': 2},
  'appliedOpTime': {'ts': Timestamp(1710969084, 1), 't': 2},
  'durableOpTime': {'ts': Timestamp(1710969084, 1), 't': 2},
  'lastAppliedWallTime': datetime.datetime(2024, 3, 20, 21, 11, 24, 949000),
  'lastDurableWallTime': datetime.datetime(2024, 3, 20, 21, 11, 24, 949000)},
 'lastStableRecoveryTimestamp': Timestamp(1710969064, 1),
 'electionCandidateMetrics': {'lastElectionReason': 'stepUpRequestSkipDryRun',
  'lastElectionDate': datetime.datetime(2024, 3, 20, 20, 51, 14, 

In [128]:
mydb = myclient["dd"]
items = mydb.get_collection('items', write_concern=pymongo.WriteConcern(w=1, wtimeout=6000))
print(items.insert_one({ 'item': 9, 'qty' : 1000, 'type': "Clasper" }))

InsertOneResult(ObjectId('65fb53f73836103bf9b342af'), acknowledged=True)


In [134]:
items_local = mydb.get_collection('items', read_concern=pymongo.read_concern.ReadConcern(level='local'))
pprint(items_local.find_one({'item': 9}))

{'_id': ObjectId('65fb53f73836103bf9b342af'),
 'item': 9,
 'qty': 1000,
 'type': 'Clasper'}


In [137]:
items_majority = mydb.get_collection('items', read_concern=pymongo.read_concern.ReadConcern(level='majority'))
pprint(items_majority.find_one({'item': 9}))

None


In [138]:
items_linearizable = mydb.get_collection('items', read_concern=pymongo.read_concern.ReadConcern(level='linearizable'))
pprint(items_linearizable.find_one({'item': 9}))

NotPrimaryError: cannot satisfy linearizable read concern on non-primary node, full error: {'topologyVersion': {'processId': ObjectId('65fb4167f6f0cf4af31b9b95'), 'counter': 16}, 'ok': 0.0, 'errmsg': 'cannot satisfy linearizable read concern on non-primary node', 'code': 10107, 'codeName': 'NotWritablePrimary', '$clusterTime': {'clusterTime': Timestamp(1710969847, 1), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1710969847, 1)}

In [140]:
# Виключаю PRIMARY і включаю СЕКОНДАРІ
myclient_1 = pymongo.MongoClient("mongodb://localhost:27017/?directConnection=true")
myclient_1.admin.command("replSetGetStatus")

{'set': 'rs0',
 'date': datetime.datetime(2024, 3, 20, 21, 29, 34, 306000),
 'myState': 2,
 'term': 5,
 'syncSourceHost': 'mongo-node3:27017',
 'syncSourceId': 2,
 'heartbeatIntervalMillis': 2000,
 'majorityVoteCount': 2,
 'writeMajorityCount': 2,
 'votingMembersCount': 3,
 'writableVotingMembersCount': 3,
 'optimes': {'lastCommittedOpTime': {'ts': Timestamp(1710970170, 1), 't': 5},
  'lastCommittedWallTime': datetime.datetime(2024, 3, 20, 21, 29, 30, 595000),
  'readConcernMajorityOpTime': {'ts': Timestamp(1710970170, 1), 't': 5},
  'appliedOpTime': {'ts': Timestamp(1710970170, 1), 't': 5},
  'durableOpTime': {'ts': Timestamp(1710970170, 1), 't': 5},
  'lastAppliedWallTime': datetime.datetime(2024, 3, 20, 21, 29, 30, 595000),
  'lastDurableWallTime': datetime.datetime(2024, 3, 20, 21, 29, 30, 595000)},
 'lastStableRecoveryTimestamp': Timestamp(1710970130, 1),
 'electionParticipantMetrics': {'votedForCandidate': True,
  'electionTerm': 5,
  'lastVoteDate': datetime.datetime(2024, 3, 20

In [143]:
# Включаю знов другу
myclient_2 = pymongo.MongoClient("mongodb://localhost:27018/?directConnection=true")
mydb_2 = myclient_2['dd']
items_local = mydb_2.get_collection('items', read_concern=pymongo.read_concern.ReadConcern(level='local'))
pprint(items_local.find_one({'item': 9}))

# Запису немає

None


### 7. Земулювати eventual consistency за допомогою установки затримки реплікації для репліки https://docs.mongodb.com/manual/tutorial/configure-a-delayed-replica-set-member/ 


In [147]:
# Знайдемо PRIMARY
myclient = pymongo.MongoClient("mongodb://localhost:27017/?directConnection=true")
myclient.admin.command("replSetGetStatus")

{'set': 'rs0',
 'date': datetime.datetime(2024, 3, 20, 22, 6, 38, 887000),
 'myState': 2,
 'term': 5,
 'syncSourceHost': 'mongo-node3:27017',
 'syncSourceId': 2,
 'heartbeatIntervalMillis': 2000,
 'majorityVoteCount': 2,
 'writeMajorityCount': 2,
 'votingMembersCount': 3,
 'writableVotingMembersCount': 3,
 'optimes': {'lastCommittedOpTime': {'ts': Timestamp(1710972390, 1), 't': 5},
  'lastCommittedWallTime': datetime.datetime(2024, 3, 20, 22, 6, 30, 698000),
  'readConcernMajorityOpTime': {'ts': Timestamp(1710972390, 1), 't': 5},
  'appliedOpTime': {'ts': Timestamp(1710972390, 1), 't': 5},
  'durableOpTime': {'ts': Timestamp(1710972390, 1), 't': 5},
  'lastAppliedWallTime': datetime.datetime(2024, 3, 20, 22, 6, 30, 698000),
  'lastDurableWallTime': datetime.datetime(2024, 3, 20, 22, 6, 30, 698000)},
 'lastStableRecoveryTimestamp': Timestamp(1710972350, 1),
 'electionParticipantMetrics': {'votedForCandidate': True,
  'electionTerm': 5,
  'lastVoteDate': datetime.datetime(2024, 3, 20, 21

In [155]:
# myclient = pymongo.MongoClient("mongodb://localhost:27019/?directConnection=true")

cfg = myclient.admin.command('replSetGetConfig')['config']

cfg['version'] += 1
cfg['members'][1]['priority'] = 0
cfg['members'][1]['hidden'] = True
cfg['members'][1]['secondaryDelaySecs'] = 30

myclient.admin.command('replSetReconfig', cfg)

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1710972844, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}},
 'operationTime': Timestamp(1710972844, 1)}

### 8. Лишити primary та secondary  для якої налаштована затримка реплікації (інший secondary відключити). Записати декілька значень. Спробувати прочитати значення з readConcern: {level: "linearizable"}
Має бути затримка поки значення не реплікуються на більшість нод


In [154]:
# Вимикаю один із секондарі
# myclient = pymongo.MongoClient("mongodb://localhost:27019/?directConnection=true")
myclient.admin.command("replSetGetStatus")

{'set': 'rs0',
 'date': datetime.datetime(2024, 3, 20, 22, 12, 11, 138000),
 'myState': 1,
 'term': 5,
 'syncSourceHost': '',
 'syncSourceId': -1,
 'heartbeatIntervalMillis': 2000,
 'majorityVoteCount': 2,
 'writeMajorityCount': 2,
 'votingMembersCount': 3,
 'writableVotingMembersCount': 3,
 'optimes': {'lastCommittedOpTime': {'ts': Timestamp(1710972700, 1), 't': 5},
  'lastCommittedWallTime': datetime.datetime(2024, 3, 20, 22, 11, 40, 711000),
  'readConcernMajorityOpTime': {'ts': Timestamp(1710972700, 1), 't': 5},
  'appliedOpTime': {'ts': Timestamp(1710972730, 1), 't': 5},
  'durableOpTime': {'ts': Timestamp(1710972730, 1), 't': 5},
  'lastAppliedWallTime': datetime.datetime(2024, 3, 20, 22, 12, 10, 712000),
  'lastDurableWallTime': datetime.datetime(2024, 3, 20, 22, 12, 10, 712000)},
 'lastStableRecoveryTimestamp': Timestamp(1710972700, 1),
 'electionCandidateMetrics': {'lastElectionReason': 'electionTimeout',
  'lastElectionDate': datetime.datetime(2024, 3, 20, 21, 27, 10, 547000)

In [158]:
# Додамо докемунтів
mydb = myclient["dd"]
items = mydb['items']

print(items.insert_many([{ 'item': 23, 'qty' : 1000, 'type': "Clasper" },
                         { 'item': 24, 'qty' : 1000, 'type': "Clasper" },
                         { 'item': 25, 'qty' : 1000, 'type': "Clasper" }]))


start_time = time.time()

items_linearizable = mydb.get_collection('items', read_concern=pymongo.read_concern.ReadConcern(level='linearizable'))
pprint(items_linearizable.find_one({'item': 23}))

print(round(time.time() - start_time, 2), ' seconds')

InsertManyResult([ObjectId('65fb60ec3836103bf9b342bd'), ObjectId('65fb60ec3836103bf9b342be'), ObjectId('65fb60ec3836103bf9b342bf')], acknowledged=True)
{'_id': ObjectId('65fb60ec3836103bf9b342bd'),
 'item': 23,
 'qty': 1000,
 'type': 'Clasper'}
30.01  seconds


In [159]:
# Як раз затримка на 30 секунд